# 1. Preparing the Environment and Loading the Data

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import xgboost as xgb



In [35]:
# Load the data
train_data = pd.read_excel('/content/train_data.xlsx')
test_data = pd.read_excel('/content/test_data.xlsx')


In [36]:
# Display the first few rows of the data
train_data.head()

customer_id transaction_date sub_grade        term home_ownership  \
0     10608026       2014-01-01        C5   36 months       MORTGAGE   
1     10235120       2014-01-01        E5   36 months       MORTGAGE   
2     10705805       2014-01-01        D2   36 months       MORTGAGE   
3     11044991       2014-01-01        B4   36 months       MORTGAGE   
4     10161054       2014-01-01        C3   60 months       MORTGAGE   

   cibil_score  total_no_of_acc  annual_inc  int_rate             purpose  \
0          665                9     70000.0     16.24  debt_consolidation   
1          660                8     65000.0     23.40    home_improvement   
2          660                7     73000.0     17.57               other   
3          690                5    118000.0     12.85  debt_consolidation   
4          665                5     63000.0     14.98  debt_consolidation   

   loan_amnt application_type  installment verification_status  account_bal  \
0       7200       Individual       253.99            Verified         4648   
1       6000       Individual       233.52     Source Verified        14051   
2       8000       Individual       287.50            Verified        14885   
3      10000       Individual       336.22     Source Verified         7542   
4      10000       Individual       237.80            Verified         6318   

   emp_length  loan_status  
0          11            0  
1          11            1  
2          11            0  
3           2            1  
4          11            0

In [37]:
test_data.head()

customer_id transaction_date sub_grade        term home_ownership  \
0     77298290       2016-04-01        C3   36 months       MORTGAGE   
1     78339737       2016-04-01        D5   36 months           RENT   
2     76302428       2016-04-01        E1   60 months           RENT   
3     74994638       2016-04-01        C4   36 months       MORTGAGE   
4     76995373       2016-04-01        D4   60 months       MORTGAGE   

   cibil_score  total_no_of_acc  annual_inc  int_rate             purpose  \
0          690                6    120000.0     13.67  debt_consolidation   
1          700                6     61277.0     19.99               house   
2          660                5     92000.0     19.99               house   
3          670                5    150000.0     14.46      small_business   
4          660                5     68000.0     18.99         credit_card   

   loan_amnt application_type  installment verification_status  account_bal  \
0      20000       Individual       680.36        Not Verified         9248   
1       9450       Individual       351.15            Verified         1929   
2      12700       Individual       336.41     Source Verified         7428   
3      25000       Individual       860.04        Not Verified         7993   
4      16000       Individual       414.97        Not Verified         8647   

   emp_length  loan_status  
0          11            1  
1           0            0  
2           3            1  
3           3            1  
4          11            1

# 2. Data Preprocessing

In [38]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Define features and target variable
X = train_data.drop(columns=["customer_id", "transaction_date", "loan_status"])
y = train_data["loan_status"]

# Identify categorical and numerical columns
categorical_cols = ["sub_grade", "term", "home_ownership", "purpose", "application_type", "verification_status"]
numerical_cols = ["cibil_score", "total_no_of_acc", "annual_inc", "int_rate", "loan_amnt", "installment", "account_bal", "emp_length"]

# Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_cols),
        ("cat", OneHotEncoder(drop="first"), categorical_cols)
    ]
)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply preprocessing pipeline to training and validation data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_val_preprocessed = preprocessor.transform(X_val)


# 3. Model Preparation

# Logistic Regression

In [42]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Predict and evaluate on validation data
y_val_pred_lr = log_reg.predict(X_val_preprocessed)
y_val_prob_lr = log_reg.predict_proba(X_val_preprocessed)[:, 1]

# Calculate metrics
lr_accuracy = accuracy_score(y_val, y_val_pred_lr)
lr_precision = precision_score(y_val, y_val_pred_lr)
lr_recall = recall_score(y_val, y_val_pred_lr)
lr_f1 = f1_score(y_val, y_val_pred_lr)
lr_auc = roc_auc_score(y_val, y_val_prob_lr)

print("Logistic Regression Metrics:")
print(f"Accuracy: {lr_accuracy}")
print(f"Precision: {lr_precision}")
print(f"Recall: {lr_recall}")
print(f"F1-Score: {lr_f1}")
print(f"ROC-AUC: {lr_auc}")



Logistic Regression Metrics:
Accuracy: 0.764434281693857
Precision: 0.784868093578895
Recall: 0.9384038564542047
F1-Score: 0.8547963028216735
ROC-AUC: 0.7352011873290764


# RandomForestClassifier

In [40]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Train Random Forest model
rf_clf = RandomForestClassifier(random_state=42, n_estimators=100)
rf_clf.fit(X_train_preprocessed, y_train)

# Predict and evaluate on validation data
y_val_pred_rf = rf_clf.predict(X_val_preprocessed)
y_val_prob_rf = rf_clf.predict_proba(X_val_preprocessed)[:, 1]

# Calculate metrics
rf_accuracy = accuracy_score(y_val, y_val_pred_rf)
rf_precision = precision_score(y_val, y_val_pred_rf)
rf_recall = recall_score(y_val, y_val_pred_rf)
rf_f1 = f1_score(y_val, y_val_pred_rf)
rf_auc = roc_auc_score(y_val, y_val_prob_rf)

print("Random Forest Metrics:")
print(f"Accuracy: {rf_accuracy}")
print(f"Precision: {rf_precision}")
print(f"Recall: {rf_recall}")
print(f"F1-Score: {rf_f1}")
print(f"ROC-AUC: {rf_auc}")


Random Forest Metrics:
Accuracy: 0.7583659469680313
Precision: 0.7862205123013061
Recall: 0.92429923227995
F1-Score: 0.8496867904915612
ROC-AUC: 0.7264359018473082


# XGBoost model

In [41]:
from xgboost import XGBClassifier

# Train XGBoost model
xgb_clf = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_preprocessed, y_train)

# Predict and evaluate on validation data
y_val_pred_xgb = xgb_clf.predict(X_val_preprocessed)
y_val_prob_xgb = xgb_clf.predict_proba(X_val_preprocessed)[:, 1]

# Calculate metrics
xgb_accuracy = accuracy_score(y_val, y_val_pred_xgb)
xgb_precision = precision_score(y_val, y_val_pred_xgb)
xgb_recall = recall_score(y_val, y_val_pred_xgb)
xgb_f1 = f1_score(y_val, y_val_pred_xgb)
xgb_auc = roc_auc_score(y_val, y_val_prob_xgb)

print("XGBoost Metrics:")
print(f"Accuracy: {xgb_accuracy}")
print(f"Precision: {xgb_precision}")
print(f"Recall: {xgb_recall}")
print(f"F1-Score: {xgb_f1}")
print(f"ROC-AUC: {xgb_auc}")


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:58:47] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Metrics:
Accuracy: 0.7646981223341102
Precision: 0.7898946942081815
Recall: 0.9285246682140094
F1-Score: 0.8536178361373273
ROC-AUC: 0.7375552704375012


# Summary

We used Logistic Regression and Random Forest models for their distinct advantages:

- **Logistic Regression**: A simple, interpretable model effective for binary classification, achieving high recall (0.938), making it ideal for minimizing false negatives.

- **Random Forest**: A robust ensemble method capable of handling complex data relationships, with competitive metrics (F1-Score: 0.850) and resistance to overfitting.

Together, they provide complementary insights into the dataset's predictive patterns.